In [ ]:
import dev.johnoreilly.common.data.remote.FantasyPremierLeagueApi
import dev.johnoreilly.common.di.initKoin
import dev.johnoreilly.common.model.Player
import kotlinx.coroutines.runBlocking
%use kandy
%use dataframe

val koin = initKoin().koin
val api = koin.get<FantasyPremierLeagueApi>()
 

In [ ]:
val staticInfo = runBlocking { api.fetchBootstrapStaticInfo() }
val eventStatus = runBlocking { api.fetchEventStatus() }


In [ ]:
val players = staticInfo.elements.sortedByDescending { it.total_points }.take(100)
val player = players[0]

In [ ]:
players.toDataFrame()["id", "web_name", "total_points", "selected_by_percent"]

In [ ]:
val upcomingFixtures = runBlocking { api.fetchUpcomingFixtures() }
upcomingFixtures.toDataFrame()

In [ ]:
val playerData = runBlocking { api.fetchPlayerData(player.id) }
val playerHistory = playerData.history
val playerHistoryDataFrame = playerHistory.toDataFrame()

In [ ]:

playerHistoryDataFrame.plot { 
    layout.title = player.web_name
    bars {
        x(round) { axis.name = "Fixture" }
        y(total_points) { axis.name = "Points" }
    }
}

In [ ]:
val fixtures =  playerHistory.map { it.round }
val totalPoints = playerHistory.map { it.total_points }
val goalsScored = playerHistory.map { it.goals_scored }
val assists = playerHistory.map { it.assists }
val dataset = dataFrameOf("fixture" to fixtures, "totalPoints" to totalPoints,
    "goalsScored" to goalsScored, "assists" to assists
).gather("totalPoints", "goalsScored", "assists").into("stat", "total")

dataset.groupBy("stat").plot {
    layout.title = player.web_name
    layout.size = 800 to 300
    bars {
        x("fixture")
        y("total")
        fillColor("stat") 
        position = Position.stack()
    }
}

In [ ]:
staticInfo.events.toDataFrame()

In [ ]:
staticInfo.teams.toDataFrame()